<a href="https://colab.research.google.com/github/yuhei1027/myrepository/blob/main/kaggle/amex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# コンペデータを格納しているパスを指定ください
PATH = "../content/drive/MyDrive/kaggle/amex"

In [3]:
# ====================================================
# Library
# ====================================================
import gc
import warnings
warnings.filterwarnings('ignore')
import scipy as sp
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
import os
import time
import itertools
from sklearn.ensemble import VotingClassifier

In [4]:
"""
if os.path.exists(f'{PATH}/test_folder')!=True:
  os.mkdir(f'{PATH}/test_folder')
for customer_id,df in tqdm(test.groupby("customer_ID")):
  df=df.reset_index(drop=True)
  df.to_csv(f"{PATH}/test_folder/test_{customer_id}.csv",index=False)
"""

'\nif os.path.exists(f\'{PATH}/test_folder\')!=True:\n  os.mkdir(f\'{PATH}/test_folder\')\nfor customer_id,df in tqdm(test.groupby("customer_ID")):\n  df=df.reset_index(drop=True)\n  df.to_csv(f"{PATH}/test_folder/test_{customer_id}.csv",index=False)\n'

In [5]:
ｄｓｆ

NameError: ignored

In [ ]:

def make_fold(df,n_fold=5,save=True,f_name="train"):
  id_map={id:np.random.randint(n_fold) for id in df["customer_ID"].unique()}
  df["fold"]=df["customer_ID"].apply(lambda x:id_map[x])

  if save:
    for i in range(n_fold):
      print(i)
      tmp_df=df[df["fold"]==i].reset_index(drop=True).copy()
      tmp_df.to_parquet(f'{PATH}/{f_name}_{str(i)}.parquet',index=False)
  
  return df

#test = pd.read_parquet(f'{PATH}/test.parquet')
#test=make_fold(test,f_name="test")


In [ ]:
# ====================================================
# Get the difference
# ====================================================

def get_difference(data, num_features):
    df1 = []
    customer_ids = []
    for customer_id, df in tqdm(data.groupby(['customer_ID'])):
        # Get the differences
        diff_df1 = df[num_features].diff(1).iloc[[-1]].values.astype(np.float32)
        # Append to lists
        df1.append(diff_df1)
        customer_ids.append(customer_id)
    # Concatenate
    df1 = np.concatenate(df1, axis = 0)
    # Transform to dataframe
    df1 = pd.DataFrame(df1, columns = [col + '_diff1' for col in df[num_features].columns])
    # Add customer id
    df1['customer_ID'] = customer_ids
    return df1

# ====================================================
# Read & preprocess data and save it to disk
# ====================================================
def read_preprocess_data():
    #trainデータを取得
    train = pd.read_parquet(f'{PATH}/train.parquet')
    #customer_IDとS_2以外の列名を取得
    features = train.drop(['customer_ID', 'S_2'], axis = 1).columns.to_list()
    #カテゴリカル特徴量
    cat_features = [
        "B_30",
        "B_38",
        "D_114",
        "D_116",
        "D_117",
        "D_120",
        "D_126",
        "D_63",
        "D_64",
        "D_66",
        "D_68",
    ]
    #連続特徴量
    num_features = [col for col in features if col not in cat_features]

    """
    print('Starting training feature engineer...')
    train_num_agg = train.groupby("customer_ID")[num_features].agg(['mean', 'std', 'min', 'max', 'last'])
    train_num_agg.columns = ['_'.join(x) for x in train_num_agg.columns]
    train_num_agg.reset_index(inplace = True)

    train_cat_agg = train.groupby("customer_ID")[cat_features].agg(['count', 'last', 'nunique'])
    train_cat_agg.columns = ['_'.join(x) for x in train_cat_agg.columns]
    train_cat_agg.reset_index(inplace = True)

    #targetを取得
    train_labels = pd.read_csv(f'{PATH}/train_labels.csv')
    # Transform float64 columns to float32
    cols = list(train_num_agg.dtypes[train_num_agg.dtypes == 'float64'].index)
    for col in tqdm(cols):
        train_num_agg[col] = train_num_agg[col].astype(np.float32)

    # Transform int64 columns to int32
    cols = list(train_cat_agg.dtypes[train_cat_agg.dtypes == 'int64'].index)
    for col in tqdm(cols):
        train_cat_agg[col] = train_cat_agg[col].astype(np.int32)

    # Get the difference
    train_diff = get_difference(train, num_features)
    train = train_num_agg.merge(train_cat_agg, how = 'inner', on = 'customer_ID').merge(train_diff, how = 'inner', on = 'customer_ID').merge(train_labels, how = 'inner', on = 'customer_ID')
    # Save files to disk
    train.to_parquet(f'{PATH}train_fe.parquet')
    del train_num_agg, train_cat_agg, train_diff,train
    gc.collect()
    """

    def df_groupby_agg(df,groupby_col,feature_cols,ops):
      flag=True
      for feature in feature_cols:
        for op in ops:
          tmp_df=df.groupby(groupby_col)[feature].agg(op)
          tmp_df.columns=[feature+"_"+op]
          
          if flag:
            flag=False
            df_agg=tmp_df.reset_index()
          else:
            df_agg=df_agg.merge(tmp_df.reset_index(), how = 'inner', on = 'customer_ID')
      return df_agg

    for i in range(5):


      test = pd.read_parquet(f'{PATH}/test_{str(i)}.parquet')
      
      print('Starting test feature engineer...')
      test_num_agg = test.groupby("customer_ID")[num_features].agg(['mean', 'std', 'min', 'max', 'last'])
      test_num_agg.columns = ['_'.join(x) for x in test_num_agg.columns]
      test_num_agg.reset_index(inplace = True)


      test_cat_agg = test.groupby("customer_ID")[cat_features].agg(['count', 'last', 'nunique'])
      test_cat_agg.columns = ['_'.join(x) for x in test_cat_agg.columns]
      test_cat_agg.reset_index(inplace = True)

      # Transform float64 columns to float32
      cols = list(test_num_agg.dtypes[test_num_agg.dtypes == 'float64'].index)
      for col in tqdm(cols):
          test_num_agg[col] = test_num_agg[col].astype(np.float32)

      # Transform int64 columns to int32
      cols = list(test_cat_agg.dtypes[test_cat_agg.dtypes == 'int64'].index)
      for col in tqdm(cols):
          test_cat_agg[col] = test_cat_agg[col].astype(np.int32)

      # Get the difference
      test_diff = get_difference(test, num_features)
      test = test_num_agg.merge(test_cat_agg, how = 'inner', on = 'customer_ID').merge(test_diff, how = 'inner', on = 'customer_ID')

       # Save files to disk
      test.to_parquet(f'{PATH}/test_fe_{str(i)}.parquet')

      del test_num_agg, test_cat_agg, test_diff,test
      gc.collect()


# Read & Preprocess Data
read_preprocess_data()

Starting test feature engineer...


  0%|          | 0/261 [00:00<?, ?it/s]

  0%|          | 0/22 [00:00<?, ?it/s]

  0%|          | 0/184539 [00:00<?, ?it/s]

In [ ]:
# ====================================================
# Library
# ====================================================
import os
import gc
import warnings
warnings.filterwarnings('ignore')
import random
import scipy as sp
import numpy as np
import pandas as pd
import joblib
import itertools
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
from itertools import combinations

# ====================================================
# Configurations
# ====================================================
class CFG:
    input_dir = '/content/data/'
    seed = 52
    n_folds = 6
    target = 'target'
    boosting_type = 'dart'
    metric = 'binary_logloss'

# ====================================================
# Seed everything
# ====================================================
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

# ====================================================
# Read data
# ====================================================
def read_data():
    train = pd.read_parquet(CFG.input_dir + 'train_fe.parquet')
    test = pd.read_parquet(CFG.input_dir + 'test_fe.parquet')
    return train, test

# ====================================================
# Amex metric
# ====================================================
def amex_metric(y_true, y_pred):
    labels = np.transpose(np.array([y_true, y_pred]))
    labels = labels[labels[:, 1].argsort()[::-1]]
    weights = np.where(labels[:,0]==0, 20, 1)
    cut_vals = labels[np.cumsum(weights) <= int(0.04 * np.sum(weights))]
    top_four = np.sum(cut_vals[:,0]) / np.sum(labels[:,0])
    gini = [0,0]
    for i in [1,0]:
        labels = np.transpose(np.array([y_true, y_pred]))
        labels = labels[labels[:, i].argsort()[::-1]]
        weight = np.where(labels[:,0]==0, 20, 1)
        weight_random = np.cumsum(weight / np.sum(weight))
        total_pos = np.sum(labels[:, 0] *  weight)
        cum_pos_found = np.cumsum(labels[:, 0] * weight)
        lorentz = cum_pos_found / total_pos
        gini[i] = np.sum((lorentz - weight_random) * weight)
    return 0.6 * (gini[1]/gini[0] + top_four)

# ====================================================
# LGBM amex metric
# ====================================================
def lgb_amex_metric(y_pred, y_true):
    y_true = y_true.get_label()
    return 'amex_metric', amex_metric(y_true, y_pred), True

# ====================================================
# Train & Evaluate
# ====================================================
def train_and_evaluate(train, test):
    # Label encode categorical features
    cat_features = [
        "B_30",
        "B_38",
        "D_114",
        "D_116",
        "D_117",
        "D_120",
        "D_126",
        "D_63",
        "D_64",
        "D_66",
        "D_68"
    ]
    cat_features = [f"{cf}_last" for cf in cat_features]
    for cat_col in cat_features:
        encoder = LabelEncoder()
        train[cat_col] = encoder.fit_transform(train[cat_col])
        test[cat_col] = encoder.transform(test[cat_col])
    # Round last float features to 2 decimal place
    num_cols = list(train.dtypes[(train.dtypes == 'float32') | (train.dtypes == 'float64')].index)
    num_cols = [col for col in num_cols if 'last' in col]
    for col in num_cols:
        train[col + '_round2'] = train[col].round(2)
        test[col + '_round2'] = test[col].round(2)
    # Get the difference between last and mean
    num_cols = [col for col in train.columns if 'last' in col]
    num_cols = [col[:-5] for col in num_cols if 'round' not in col]
    for col in num_cols:
        try:
            train[f'{col}_last_mean_diff'] = train[f'{col}_last'] - train[f'{col}_mean']
            test[f'{col}_last_mean_diff'] = test[f'{col}_last'] - test[f'{col}_mean']
        except:
            pass
    # Transform float64 and float32 to float16
    num_cols = list(train.dtypes[(train.dtypes == 'float32') | (train.dtypes == 'float64')].index)
    for col in tqdm(num_cols):
        train[col] = train[col].astype(np.float16)
        test[col] = test[col].astype(np.float16)
    # Get feature list
    features = [col for col in train.columns if col not in ['customer_ID', CFG.target]]
    params = {
        'objective': 'binary',
        'metric': CFG.metric,
        'boosting': CFG.boosting_type,
        'seed': CFG.seed,
        'num_leaves': 100,
        'learning_rate': 0.013,
        'feature_fraction': 0.20,
        'bagging_freq': 10.56,
        'bagging_fraction': 0.55,
        'n_jobs': -1,
        'lambda_l2': 2,
        'min_data_in_leaf': 100,
        }
    score =0
    test=[]
    # Create a numpy array to store test predictions
    test_predictions = np.zeros(len(test))
    # Create a numpy array to store out of folds predictions
    oof_predictions = np.zeros(len(train))
    kfold = StratifiedKFold(n_splits = CFG.n_folds, shuffle = True, random_state = CFG.seed)
    for fold, (trn_ind, val_ind) in enumerate(kfold.split(train, train[CFG.target])):
        print(' ')
        print('-'*50)
        print(f'Training fold {fold} with {len(features)} features...')
        x_train, x_val = train[features].iloc[trn_ind], train[features].iloc[val_ind]
        y_train, y_val = train[CFG.target].iloc[trn_ind], train[CFG.target].iloc[val_ind]
        lgb_train = lgb.Dataset(x_train, y_train, categorical_feature = cat_features)
        lgb_valid = lgb.Dataset(x_val, y_val, categorical_feature = cat_features)
        model = lgb.train(
        params = params,
        train_set = lgb_train,
        num_boost_round = 11000,
        valid_sets = [lgb_train, lgb_valid],
        early_stopping_rounds = 1500,
            verbose_eval = 500,
            feval = lgb_amex_metric
            )
        # Save best model
        joblib.dump(model, f'/content/drive/MyDrive/Amex/Models/lgbm_{CFG.boosting_type}_fold{fold}_seed{CFG.seed}.pkl')
        # Predict validation
        val_pred = model.predict(x_val)
        # Add to out of folds array
        oof_predictions[val_ind] = val_pred
        # Predict the test set
        test_pred = model.predict(test[features])
        test_predictions += test_pred / CFG.n_folds
        # Compute fold metric
        score = amex_metric(y_val, val_pred)
        print(f'Our fold {fold} CV score is {score}')
        del x_train, x_val, y_train, y_val, lgb_train, lgb_valid
        gc.collect()
    # Compute out of folds metric
        score = amex_metric(train[CFG.target], oof_predictions)
        print(f'Our out of folds CV score is {score}')
     
    # Create a dataframe to store out of folds predictions
        oof_df = pd.DataFrame({'customer_ID': train['customer_ID'], 'target': train[CFG.target], 'prediction': oof_predictions})
        oof_df.to_csv(f'/content/drive/MyDrive/Amex/OOF/oof_lgbm_{CFG.boosting_type}_baseline_{CFG.n_folds}fold_seed{CFG.seed}.csv', index = False)
    # Create a dataframe to store test prediction
        test_df = pd.DataFrame({'customer_ID': test['customer_ID'], 'prediction': test_predictions})
        test_df.to_csv(f'/content/drive/MyDrive/Amex/Predictions/test_lgbm_{CFG.boosting_type}_baseline_{CFG.n_folds}fold_seed{CFG.seed}.csv', index = False)

# seed_everything(CFG.seed)
# train, test = read_data()
# train_and_evaluate(train, test)
# ====================================================
# Library
# ====================================================
        import os
        import gc
        import joblib
        import random
        import warnings
        import itertools
        import scipy as sp
        import numpy as np
        import pandas as pd
        from tqdm import tqdm
        import xgboost as xgb
        import lightgbm as lgb
        warnings.filterwarnings('ignore')
        from itertools import combinations
        pd.set_option('display.width', 1000)
        pd.set_option('display.max_rows', 500)
        from catboost import CatBoostClassifier
        pd.set_option('display.max_columns', 500)
        from sklearn.preprocessing import LabelEncoder
        from sklearn.model_selection import StratifiedKFold, train_test_split

        class CFG:
            input_dir = '../input/amex-fe/'
            seed = 52
            n_folds = 6
            target = 'target'

        def seed_everything(seed):
            random.seed(seed)
            np.random.seed(seed)
            os.environ['PYTHONHASHSEED'] = str(seed)

        def read_data():
            train = pd.read_parquet(CFG.input_dir + 'train_fe_plus_plus.parquet')
            test = pd.read_parquet(CFG.input_dir + 'test_fe_plus_plus.parquet')
            return train, test

        def amex_metric(y_true, y_pred):
            labels = np.transpose(np.array([y_true, y_pred]))
            labels = labels[labels[:, 1].argsort()[::-1]]
            weights = np.where(labels[:,0]==0, 20, 1)
            cut_vals = labels[np.cumsum(weights) <= int(0.04 * np.sum(weights))]
            top_four = np.sum(cut_vals[:,0]) / np.sum(labels[:,0])
            gini = [0,0]
            for i in [1,0]:
                labels = np.transpose(np.array([y_true, y_pred]))
                labels = labels[labels[:, i].argsort()[::-1]]
                weight = np.where(labels[:,0]==0, 20, 1)
                weight_random = np.cumsum(weight / np.sum(weight))
                total_pos = np.sum(labels[:, 0] *  weight)
                cum_pos_found = np.cumsum(labels[:, 0] * weight)
                lorentz = cum_pos_found / total_pos
                gini[i] = np.sum((lorentz - weight_random) * weight)
            return 0.5 * (gini[1]/gini[0] + top_four)
        g=0

        def amex_metric_np(preds, target):
            indices = np.argsort(preds)[::-1]
            preds, target = preds[indices], target[indices]
            weight = 20.0 - target * 19.0
            cum_norm_weight = (weight / weight.sum()).cumsum()
            four_pct_mask = cum_norm_weight <= 0.04
            d = np.sum(target[four_pct_mask]) / np.sum(target)
            weighted_target = target * weight
            lorentz = (weighted_target / weighted_target.sum()).cumsum()
            gini = ((lorentz - cum_norm_weight) * weight).sum()
            n_pos = np.sum(target)
            n_neg = target.shape[0] - n_pos
            gini_max = 10 * n_neg * (n_pos + 20 * n_neg - 19) / (n_pos + 20 * n_neg)
            g = gini / gini_max
            return 0.6 * (g + d)
            N_FOLDS = 5
    skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=22)
    y_oof = np.zeros(train_x.shape[0])
    y_test = np.zeros(test.shape[0])
    ix = 0
    for train_ind, val_ind in skf.split(train_x, train_y):
        print(f"******* Fold {ix} ******* ")
        tr_x, val_x = (
            train_x.iloc[train_ind].reset_index(drop=True),
            train_x.iloc[val_ind].reset_index(drop=True),
    )
        tr_y, val_y = (
            train_y.iloc[train_ind].reset_index(drop=True),
            train_y.iloc[val_ind].reset_index(drop=True),
    )

        clf = CatBoostClassifier(iterations=5000, random_state=22)
        clf.fit(tr_x, tr_y, eval_set=[(val_x, val_y)], cat_features=cat_features,  verbose=100)
        preds = clf.predict_proba(val_x)[:, 1]
        y_oof[val_ind] = y_oof[val_ind] + preds

        preds_test = clf.predict_proba(test)[:, 1]
        y_test = y_test + preds_test / N_FOLDS
        ix = ix + 1
    y_pred = train_y.copy(deep=True)
    y_pred = y_pred.rename(columns={"target": "prediction"})
    y_pred["prediction"] = y_oof
    val_score = amex_metric(train_y, y_pred)
    print(f"Amex metric: {val_score}")
import os
import gc
import warnings
warnings.filterwarnings('ignore')
import random
import scipy as sp
import numpy as np
import pandas as pd
import joblib
import itertools
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
from itertools import combinations

# ====================================================
# Configurations
# ====================================================
class CFG:
    input_dir = '/content/data/'
    seed = 42
    n_folds = 5
    target = 'target'

# ====================================================
# Seed everything
# ====================================================
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

# ====================================================
# Read data
# ====================================================
def read_data():
    train = pd.read_parquet(CFG.input_dir + 'train_fe.parquet')
    test = pd.read_parquet(CFG.input_dir + 'test_fe.parquet')
    return train, test

# ====================================================
# Amex metric
# ====================================================
def amex_metric(y_true, y_pred):
    labels = np.transpose(np.array([y_true, y_pred]))
    labels = labels[labels[:, 1].argsort()[::-1]]
    weights = np.where(labels[:,0]==0, 20, 1)
    cut_vals = labels[np.cumsum(weights) <= int(0.04 * np.sum(weights))]
    top_four = np.sum(cut_vals[:,0]) / np.sum(labels[:,0])
    gini = [0,0]
    for i in [1,0]:
        labels = np.transpose(np.array([y_true, y_pred]))
        labels = labels[labels[:, i].argsort()[::-1]]
        weight = np.where(labels[:,0]==0, 20, 1)
        weight_random = np.cumsum(weight / np.sum(weight))
        total_pos = np.sum(labels[:, 0] *  weight)
        cum_pos_found = np.cumsum(labels[:, 0] * weight)
        lorentz = cum_pos_found / total_pos
        gini[i] = np.sum((lorentz - weight_random) * weight)
    return 0.5 * (gini[1]/gini[0] + top_four)

# ====================================================
# LGBM amex metric
# ====================================================
def lgb_amex_metric(y_pred, y_true):
    y_true = y_true.get_label()
    return 'amex_metric', amex_metric(y_true, y_pred), True

# ====================================================
# Train & Evaluate
# ====================================================
def train_and_evaluate(train, test):
    # Label encode categorical features
    cat_features = [
        "B_30",
        "B_38",
        "D_114",
        "D_116",
        "D_117",
        "D_120",
        "D_126",
        "D_63",
        "D_64",
        "D_66",
        "D_68"
    ]
    cat_features = [f"{cf}_last" for cf in cat_features]
    for cat_col in cat_features:
        encoder = LabelEncoder()
        train[cat_col] = encoder.fit_transform(train[cat_col])
        test[cat_col] = encoder.transform(test[cat_col])
    # Round last float features to 2 decimal place
    num_cols = list(train.dtypes[(train.dtypes == 'float32') | (train.dtypes == 'float64')].index)
    num_cols = [col for col in num_cols if 'last' in col]
    for col in num_cols:
        train[col + '_round2'] = train[col].round(2)
        test[col + '_round2'] = test[col].round(2)
    # Get feature list
    features = [col for col in train.columns if col not in ['customer_ID', CFG.target]]
    params = {
        'objective': 'binary',
        'metric': "binary_logloss",
        'boosting': 'dart',
        'seed': CFG.seed,
        'num_leaves': 100,
        'learning_rate': 0.01,
        'feature_fraction': 0.20,
        'bagging_freq': 10,
        'bagging_fraction': 0.50,
        'n_jobs': -1,
        'lambda_l2': 2,
        'min_data_in_leaf': 40
        }
    # Create a numpy array to store test predictions
    test_predictions = np.zeros(len(test))
    # Create a numpy array to store out of folds predictions
    oof_predictions = np.zeros(len(train))
    kfold = StratifiedKFold(n_splits = CFG.n_folds, shuffle = True, random_state = CFG.seed)
    for fold, (trn_ind, val_ind) in enumerate(kfold.split(train, train[CFG.target])):
        print(' ')
        print('-'*50)
        print(f'Training fold {fold} with {len(features)} features...')
        x_train, x_val = train[features].iloc[trn_ind], train[features].iloc[val_ind]
        y_train, y_val = train[CFG.target].iloc[trn_ind], train[CFG.target].iloc[val_ind]
        lgb_train = lgb.Dataset(x_train, y_train, categorical_feature = cat_features)
        lgb_valid = lgb.Dataset(x_val, y_val, categorical_feature = cat_features)
        model = lgb.train(
            params = params,
            train_set = lgb_train,
            num_boost_round = 10500,
            valid_sets = [lgb_train, lgb_valid],
            early_stopping_rounds = 100,
            verbose_eval = 500,
            feval = lgb_amex_metric
            )
        # Save best model
        joblib.dump(model, f'/content/drive/MyDrive/Amex/Models/lgbm_fold{fold}_seed{CFG.seed}.pkl')
        # Predict validation
        val_pred = model.predict(x_val)
        # Add to out of folds array
        oof_predictions[val_ind] = val_pred
        # Predict the test set
        test_pred = model.predict(test[features])
        test_predictions += test_pred / CFG.n_folds
        # Compute fold metric
        score = amex_metric(y_val, val_pred)
        print(f'Our fold {fold} CV score is {score}')
        del x_train, x_val, y_train, y_val, lgb_train, lgb_valid
        gc.collect()
    # Compute out of folds metric
    score = amex_metric(train[CFG.target], oof_predictions)
    print(f'Our out of folds CV score is {score}')
    # Create a dataframe to store out of folds predictions
    oof_df = pd.DataFrame({'customer_ID': train['customer_ID'], 'target': train[CFG.target], 'prediction': oof_predictions})
    oof_df.to_csv(f'/content/drive/MyDrive/Amex/OOF/oof_lgbm_baseline_{CFG.n_folds}fold_seed{CFG.seed}.csv', index = False)
    # Create a dataframe to store test prediction
    test_df = pd.DataFrame({'customer_ID': test['customer_ID'], 'prediction': test_predictions})
    test_df.to_csv(f'/content/drive/MyDrive/Amex/Predictions/test_lgbm_baseline_{CFG.n_folds}fold_seed{CFG.seed}.csv', index = False)

    ensemble=VotingClassifier(estimators=[('Decision Tree',g ), ('Random Forest', score)], voting='soft', weights=[2,1]).fit(train_x,train_y)
    print('The accuracy for DecisionTree and Random Forest is:',ensemble.score(test_x,test_y))
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.model_selection import train_test_split
    train_x, test_x, train_y, test_y = train_test_split(dataset.drop('target'),dataset['target'])
    trained_model = random_forest_classifier(train_x, train_y)
    predictions = trained_model.predict(test_x)
        
